In [13]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import matplotlib.pyplot as plt, argparse, numpy as np, math, sys, copy
from skimage.segmentation import slic, mark_boundaries, felzenszwalb
from skimage.util import img_as_float
from skimage import io
import numpy as np
import superpixels as sp
import glob
import pipeline as pipe
import time
import superpixels as sp
import scipy.stats as sci
from PIL import Image
from scipy import ndimage
import cv2

In [15]:
import argparse
import LoadBatches
from Models import Segnet, VGGSegnet, VGGUnet, FCN8, FCN32
from keras.models import load_model
import glob
import cv2
import numpy as np
import random

### Parameters

In [41]:
#weights
save_weights_path = '/home/falreis/Me/mestrado/ppginf-disciplinas/deep_learning/tp_final/image-segmentation-keras/code/weights/'

#input and output images
images_path="/media/falreis/falreis1/mestrado/deep_learning/BSDS500/pre_process/images/test/"  
output_path='/media/falreis/falreis1/mestrado/deep_learning/BSDS500/predict/'

#input height and width
input_height = 481
input_width = 481

#classes and epochs
n_classes = 2
epoch_number = 3

#optmizer and model name
optimizer_name = 'adam'
model_name = 'segnet'

### Predict

In [42]:
modelFns = { 'segnet':Segnet.Segnet
            , 'vgg_segnet':VGGSegnet.VGGSegnet
            , 'vgg_unet':VGGUnet.VGGUnet
            , 'vgg_unet2':VGGUnet.VGGUnet2
            , 'fcn8':FCN8.FCN8
            , 'fcn32':FCN32.FCN32 
}
modelFN = modelFns[ model_name ]

m = modelFN( n_classes , input_height=input_height, input_width=input_width)
m.load_weights(save_weights_path + "." + str(epoch_number))
m.compile(loss='categorical_crossentropy',
      optimizer= optimizer_name ,
      metrics=['accuracy'])

output_height = m.outputHeight
output_width = m.outputWidth

images = glob.glob( images_path + "*.jpg"  ) + glob.glob( images_path + "*.png"  ) +  glob.glob( images_path + "*.jpeg"  )
images.sort()

colors = [  ( random.randint(0,255),random.randint(0,255),random.randint(0,255)   ) for _ in range(n_classes)  ]

for imgName in images:
    outName = imgName.replace(images_path, output_path )
    X = LoadBatches.getImageArr(imgName, input_width, input_height)
    pr = m.predict( np.array([X]) )[0]
    pr = pr.reshape(( output_height ,  output_width , n_classes ) ).argmax( axis=2 )
    seg_img = np.zeros( ( output_height , output_width , 3  ) )
    
    if(len(np.unique(pr)) > 1):
        print(np.unique(pr))
    
    for c in range(n_classes):
        seg_img[:,:,0] += ( (pr[:,: ] == c )*( 255 )).astype('uint8')
        seg_img[:,:,1] += ((pr[:,: ] == c )*( 255 )).astype('uint8')
        seg_img[:,:,2] += ((pr[:,: ] == c )*( 255 )).astype('uint8')
    seg_img = cv2.resize(seg_img  , (input_width , input_height ))
    cv2.imwrite(  outName , seg_img )
    
print('Done')

Done
